In [ ]:
import org.apache.spark.sql.{DataFrame, Encoders, SparkSession}
import org.apache.spark.sql.types.StructType
import org.bdgenomics.adam.rdd.ADAMContext._
import org.bdgenomics.adam.rdd.ADAMContextExtensions._
import scala.reflect.runtime.universe._
import comp.bio.aging.playground.extras.uniprot._
import org.apache.spark.storage.StorageLevel

import org.apache.spark.sql.{DataFrame, Encoders, SparkSession}
import org.apache.spark.sql.types.StructType
import org.bdgenomics.adam.rdd.ADAMContext._
import org.bdgenomics.adam.rdd.ADAMContextExtensions._
import scala.reflect.runtime.universe._
import comp.bio.aging.playground.extras.uniprot._
import org.apache.spark.storage.StorageLevel


In [ ]:
val spark = SparkSession
  .builder()
  .appName("annotations")
  .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@aa5e9dc


1. UniProtKB-AC
2. UniProtKB-ID
3. GeneID (EntrezGene)
4. RefSeq
5. GI
6. PDB
7. GO
8. UniRef100
9. UniRef90
10. UniRef50
11. UniParc
12. PIR
13. NCBI-taxon
14. MIM
15. UniGene
16. PubMed
17. EMBL
18. EMBL-CDS
19. Ensembl
20. Ensembl_TRS
21. Ensembl_PRO
22. Additional PubMed

In [ ]:
val base = "hdfs://namenode/pipelines"
val diamond = base + "/diamond"
val blastp = diamond + "/blastp"
val graywhale = base + "/GRAY_WHALE"
val trinity = graywhale + "/Trin_Mitya.Trinity.fasta"
val proteins = graywhale + "/Trin_Mitya.Trinity.fasta.transdecoder.pep"
val gff = graywhale + "/Trin_Mitya.Trinity.fasta.transdecoder.gff3"

val proteinsAdam = graywhale + "/proteins.adam"
val transcriptsAdam = graywhale + "/transcripts.adam"
val transcripts = sparkContext.loadParquetContigFragments(transcriptsAdam)
val query = sparkContext.loadParquetContigFragments(proteinsAdam)


base: String = hdfs://namenode/pipelines
diamond: String = hdfs://namenode/pipelines/diamond
blastp: String = hdfs://namenode/pipelines/diamond/blastp
graywhale: String = hdfs://namenode/pipelines/GRAY_WHALE
trinity: String = hdfs://namenode/pipelines/GRAY_WHALE/Trin_Mitya.Trinity.fasta
proteins: String = hdfs://namenode/pipelines/GRAY_WHALE/Trin_Mitya.Trinity.fasta.transdecoder.pep
gff: String = hdfs://namenode/pipelines/GRAY_WHALE/Trin_Mitya.Trinity.fasta.transdecoder.gff3
proteinsAdam: String = hdfs://namenode/pipelines/GRAY_WHALE/proteins.adam
transcriptsAdam: String = hdfs://namenode/pipelines/GRAY_WHALE/transcripts.adam
transcripts: org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD = ParquetUnboundNucleotideContigFragmentRDD with 114233 reference sequences
query: org.bdge...

In [ ]:
val predictions = query.rdd
  .map(q=>comp.bio.aging.playground.extras.diamond.ProteinPrediction.extractPrediction(q.getDescription, q.getSequence, q.getContigName))
  .filter(p=>p.diamondHits.nonEmpty)
  //.filter(p=>p.diamondHits.nonEmpty || p.pfamHits.nonEmpty)
  .map(p=>(p.transcript, p))

predictions: org.apache.spark.rdd.RDD[(String, comp.bio.aging.playground.extras.diamond.ProteinPrediction)] = MapPartitionsRDD[4] at map at <console>:91


In [ ]:
predictions.take(10)

res5: Array[(String, comp.bio.aging.playground.extras.diamond.ProteinPrediction)] = Array((c47981_g1_i1,ProteinPrediction(c47981_g1_i1,g.54638,EKMRIGRAVSSLLIQHVKNTVKTFEELLSGNKIDYYMEQPFREQLAKGSQLAESLASKFNTDGCISKKNQRQQVLRNLSILRELQKKGKMTEVLETKQDA,internal,33.33,1,300,101,-,List(BlastResult(UniRef90_F1SDF7,70.8,9.2e-27), BlastResult(UniRef90_UPI0007DC515E,69.0,1.4e-27), BlastResult(UniRef90_UPI0005F4B8C9,70.0,1.1e-27), BlastResult(UniRef90_G3TXE7,72.0,1.1e-27), BlastResult(UniRef90_UPI0004F1F2C2,70.0,2.8e-28), BlastResult(UniRef90_A0A1D5QB44,70.0,2.8e-28), BlastResult(UniRef90_G1LK58,72.3,5.7e-29), BlastResult(UniRef90_H0Y0T9,76.2,1.5e-29), BlastResult(UniRef90_UPI0003EDE850,71.4,1.2e-29), BlastResult(UniRef90_UPI00064382AA,76.0,3.0e-30), BlastResult(UniRef90_D2I655,73.0,2.3e-30), BlastResu...

entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
predictions.first

res7: (String, comp.bio.aging.playground.extras.diamond.ProteinPrediction) = (c47981_g1_i1,ProteinPrediction(c47981_g1_i1,g.54638,EKMRIGRAVSSLLIQHVKNTVKTFEELLSGNKIDYYMEQPFREQLAKGSQLAESLASKFNTDGCISKKNQRQQVLRNLSILRELQKKGKMTEVLETKQDA,internal,33.33,1,300,101,-,List(BlastResult(UniRef90_F1SDF7,70.8,9.2e-27), BlastResult(UniRef90_UPI0007DC515E,69.0,1.4e-27), BlastResult(UniRef90_UPI0005F4B8C9,70.0,1.1e-27), BlastResult(UniRef90_G3TXE7,72.0,1.1e-27), BlastResult(UniRef90_UPI0004F1F2C2,70.0,2.8e-28), BlastResult(UniRef90_A0A1D5QB44,70.0,2.8e-28), BlastResult(UniRef90_G1LK58,72.3,5.7e-29), BlastResult(UniRef90_H0Y0T9,76.2,1.5e-29), BlastResult(UniRef90_UPI0003EDE850,71.4,1.2e-29), BlastResult(UniRef90_UPI00064382AA,76.0,3.0e-30), BlastResult(UniRef90_D2I655,73.0,2.3e-30), BlastResult(UniRef90_U...

(c47981_g1_i1,ProteinPrediction(c47981_g1_i1,g.54638,EKMRIGRAVSSLLIQHVKNTVKTFEELLSGNKIDYYMEQPFREQLAKGSQLAESLASKFNTDGCISKKNQRQQVLRNLSILRELQKKGKMTEVLETKQDA,internal,33.33,1,300,101,-,List(BlastResult(UniRef90_F1SDF7,70.8,9.2e-27), BlastResult(UniRef90_UPI0007DC515E,69.0,1.4e-27), BlastResult(UniRef90_UPI0005F4B8C9,70.0,1.1e-27), BlastResult(UniRef90_G3TXE7,72.0,1.1e-27), BlastResult(UniRef90_UPI0004F1F2C2,70.0,2.8e-28), BlastResult(UniRef90_A0A1D5QB44,70.0,2.8e-28), BlastResult(UniRef90_G1LK58,72.3,5.7e-29), BlastResult(UniRef90_H0Y0T9,76.2,1.5e-29), BlastResult(UniRef90_UPI0003EDE850,71.4,1.2e-29), BlastResult(UniRef90_UPI00064382AA,76.0,3.0e-30), BlastResult(UniRef90_D2I655,73.0,2.3e-30), BlastResult(UniRef90_UPI000343FCE3,74.0,1.8e-30), BlastResult(UniRef90_M3Y2M2,74.0,1.8e-30), BlastResult(UniRef90_UPI0009047864,74.0,1.4e-30), BlastResult(UniRef90_UPI0002C59059,71.0,1.4e-30), BlastResult(UniRef90_UPI0002C67C93,75.0,1.0e-30), BlastResult(UniRef90_UPI0009485D0D,75.0,3.6e-31), BlastResult(UniRef90_M3WYX1,75.0,3.6e-31), BlastResult(UniRef90_A0A212CF79,78.0,5.5e-32), BlastResult(UniRef90_UPI000BAA1BCB,74.0,1.9e-32), BlastResult(UniRef90_A5PJD2,79.0,1.5e-32), BlastResult(UniRef90_UPI00070458C8,83.7,5.4e-35), BlastResult(UniRef90_UPI00057B7029,83.0,8.3e-36), BlastResult(UniRef90_S9W8A8,83.0,8.3e-36), BlastResult(UniRef90_UPI000952500E,93.9,1.1e-40)),List(),Gene.54638::c47981_g1_i1::g.54638::m.54638))

In [ ]:
val mapping =  spark.readTypedTSV[UniprotMapping]("/pipelines/uniprot/idmapping_selected.tab")

mapping: org.apache.spark.sql.Dataset[comp.bio.aging.playground.extras.uniprot.UniprotMapping] = [uniprot_ac: string, uniprot_id: string ... 20 more fields]


In [ ]:
mapping.take(4)

res10: Array[comp.bio.aging.playground.extras.uniprot.UniprotMapping] = Array(UniprotMapping(Q6GZX4,001R_FRG3G,2947773,YP_031579.1,81941549; 49237298,null,GO:0006355; GO:0046782; GO:0006351,UniRef100_Q6GZX4,UniRef90_Q6GZX4,UniRef50_Q6GZX4,UPI00003B0FD4,null,654924,null,null,15165820,AY548484,AAT09660.1,null,null,null,null), UniprotMapping(Q6GZX3,002L_FRG3G,2947774,YP_031580.1,49237299; 81941548; 47060117,null,GO:0033644; GO:0016021,UniRef100_Q6GZX3,UniRef90_Q6GZX3,UniRef50_Q6GZX3,UPI00003B0FD5,null,654924,null,null,15165820,AY548484,AAT09661.1,null,null,null,null), UniprotMapping(Q197F8,002R_IIV3,4156251,YP_654574.1,109287880; 123808694; 106073503,null,null,UniRef100_Q197F8,UniRef90_Q197F8,UniRef50_Q197F8,UPI0000D83464,null,345201,null,null,16912294,DQ643392,ABF82032.1,null,null,null,nu...

entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
val features = sparkContext.loadGff3(gff)

features: org.bdgenomics.adam.rdd.feature.FeatureRDD = RDDBoundFeatureRDD with 0 reference sequences


In [ ]:
import org.apache.spark.storage.StorageLevel
val byFeature = features.rdd.map(f=> (f.getFeatureType , f)).persist(StorageLevel.MEMORY_AND_DISK)

import org.apache.spark.storage.StorageLevel
byFeature: org.apache.spark.rdd.RDD[(String, org.bdgenomics.formats.avro.Feature)] = MapPartitionsRDD[11] at map at <console>:89


In [ ]:
query.rdd.take(10)

res14: Array[org.bdgenomics.formats.avro.NucleotideContigFragment] = Array({"contigName": "Gene.54638::c47981_g1_i1::g.54638::m.54638", "contigLength": 100, "description": "Gene.54638::c47981_g1_i1::g.54638  ORF type:internal len:101 (-),score=33.33,UniRef90_UPI000952500E|93.9|1.1e-40,UniRef90_S9W8A8|83.0|8.3e-36,UniRef90_UPI00057B7029|83.0|8.3e-36,UniRef90_UPI00070458C8|83.7|5.4e-35,UniRef90_A5PJD2|79.0|1.5e-32,UniRef90_UPI000BAA1BCB|74.0|1.9e-32,UniRef90_A0A212CF79|78.0|5.5e-32,UniRef90_M3WYX1|75.0|3.6e-31,UniRef90_UPI0009485D0D|75.0|3.6e-31,UniRef90_UPI0002C67C93|75.0|1.0e-30,UniRef90_UPI0002C59059|71.0|1.4e-30,UniRef90_UPI0009047864|74.0|1.4e-30,UniRef90_M3Y2M2|74.0|1.8e-30,UniRef90_UPI000343FCE3|74.0|1.8e-30,UniRef90_D2I655|73.0|2.3e-30,UniRef90_UPI00064382AA|76.0|3.0e-30,UniRef90_...

entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
byFeature.countByKey()

res16: scala.collection.Map[String,Long] = Map(five_prime_UTR -> 26740, three_prime_UTR -> 28766, exon -> 32429, CDS -> 32429, mRNA -> 32429, gene -> 32429)


Map(five_prime_UTR -> 26740, three_prime_UTR -> 28766, exon -> 32429, CDS -> 32429, mRNA -> 32429, gene -> 32429)

In [ ]:
val predictionsByType = predictions.map(p=>p._2.orf_type->p).persist(StorageLevel.MEMORY_AND_DISK)
predictionsByType.countByKey()


predictionsByType: org.apache.spark.rdd.RDD[(String, (String, comp.bio.aging.playground.extras.diamond.ProteinPrediction))] = MapPartitionsRDD[14] at map at <console>:90
res18: scala.collection.Map[String,Long] = Map(internal -> 6447, complete -> 10929, 5prime_partial -> 8240, 3prime_partial -> 2445)


Map(internal -> 6447, complete -> 10929, 5prime_partial -> 8240, 3prime_partial -> 2445)

In [ ]:
val complete = predictions.filter(p => p._2.orf_type == "complete").persist(StorageLevel.MEMORY_AND_DISK)
complete.count

complete: org.apache.spark.rdd.RDD[(String, comp.bio.aging.playground.extras.diamond.ProteinPrediction)] = MapPartitionsRDD[17] at filter at <console>:90
res20: Long = 10929


10929

In [ ]:
import spark.implicits._
val completeDF = complete.map{ case (tr, c) =>(tr, c.diamondHits.maxBy(r=>r.score).id, c.sequence)}.toDF("transcript", "uniref","sequence")
  

import spark.implicits._
completeDF: org.apache.spark.sql.DataFrame = [transcript: string, uniref: string ... 1 more field]


In [ ]:
val mappingDF = mapping.toDF().select("uniprot_ac",	"uniprot_id",	"entrez",	"refSeq",	"gi",	"go",	"uniref90",	"uniparc",	"taxon", "pubmed", "embl")
mappingDF.rdd.take(5)

mappingDF: org.apache.spark.sql.DataFrame = [uniprot_ac: string, uniprot_id: string ... 9 more fields]
res23: Array[org.apache.spark.sql.Row] = Array([Q6GZX4,001R_FRG3G,2947773,YP_031579.1,81941549; 49237298,GO:0006355; GO:0046782; GO:0006351,UniRef90_Q6GZX4,UPI00003B0FD4,654924,15165820,AY548484], [Q6GZX3,002L_FRG3G,2947774,YP_031580.1,49237299; 81941548; 47060117,GO:0033644; GO:0016021,UniRef90_Q6GZX3,UPI00003B0FD5,654924,15165820,AY548484], [Q197F8,002R_IIV3,4156251,YP_654574.1,109287880; 123808694; 106073503,null,UniRef90_Q197F8,UPI0000D83464,345201,16912294,DQ643392], [Q197F7,003L_IIV3,4156252,YP_654575.1,106073504; 109287881; 123808693,null,UniRef90_Q197F7,UPI0000D83465,345201,16912294,DQ643392], [Q6GZX2,003R_FRG3G,2947775,YP_031581.1,47060118; 81941547; 49237300,null,UniRef90_Q6G...

entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
val completedMapped =mappingDF.join(completeDF, completeDF("uniref") === mappingDF("uniref90")).cache()
completedMapped.rdd.take(10)


completedMapped: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [uniprot_ac: string, uniprot_id: string ... 12 more fields]
res25: Array[org.apache.spark.sql.Row] = Array([A0A061I3I4,A0A061I3I4_CRIGR,null,null,537182198,GO:0005524; GO:0008353,UniRef90_A0A061I3I4,UPI00049CA7C3,10029,23929341,KE674888,c14959_g1_i2,UniRef90_A0A061I3I4,MVKSLLYQILDGIHYLHANWVLHRDLKPANILVMGEGPERGRVKIADMGFARLFNSPLKPLADLDPVVVTFWYRAPELLLGARHYTKAIDIWAIGCIFAELLTSEPIFHCRQEDIKTSNPYHHDQLDRIFNVMGFPADKDWEDIKKMPEHSTLMKDFRRNTYTNCSLIKYMEKHKVKPDSKAFHLLQKLLTMDPIKRITSEQAMQDPYFLEDPLPTSDVFAGCQIPYPKREFLTEEEPDDKGDKNQQQQQGNNHTNGTGHPGNQDGSHTQGPPLKKVRVVPPTTTSGGLIMTSDYQRSNAHAAYPNPGPSTSQPQSSMGYSATSQQPPQYSHQTHRY], [F7AFD0,F7AFD0_XENTR,null,null,null,GO:0005524; GO:0004672,UniRef90_A0A061I3I4,UPI0002068B92,8364,20431018,AAMC01...

entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
case class MappedJoined(transcript: String, uniref90: String, taxon: String, pubmed: String, go: String, entrez: String, uniprot_id: String)
  {
    self =>

    lazy val pubmeds: Set[String] = pubmed.split(";").toSet
    lazy val goes: Set[String] = go.split(";").toSet
    lazy val taxons: Set[String] = taxon.split(";").toSet

    def merge(other: MappedJoined): MappedJoined = other.copy(
      self.taxon + ";" + other.taxon,
      self.pubmed + ";" + other.pubmed,
      self.go + ";" + other.go
    )
  }


defined class MappedJoined


In [ ]:
val smallJoined= joined.rdd.map(r=>
    ( r.getAs[String]("transcript"),
        MappedJoined(
            r.getAs[String]("transcript"),
            r.getAs[String]("uniref90"),
            r.getAs[String]("taxon"),
            r.getAs[String]("pubmed"),
            r.getAs[String]("go"),
            r.getAs[String]("entrez"),
            r.getAs[String]("uniprot_id")
        )
      )
    ).reduceByKey((a, b)=> a.merge(b)).persist(StorageLevel.MEMORY_AND_DISK) 
  

smallJoined: org.apache.spark.rdd.RDD[(String, MappedJoined)] = ShuffledRDD[57] at reduceByKey at <console>:119


In [ ]:
(completeDF.count, joined.count, smallJoined.count)

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 28.0 failed 4 times, most recent failure: Lost task 0.3 in stage 28.0 (TID 437, 10.0.2.3, executor 0): java.io.IOException: org.apache.spark.SparkException: Failed to get broadcast_25_piece0 of broadcast_25
	at org.apache.spark.util.Utils$.tryOrIOException(Utils.scala:1310)
	at org.apache.spark.broadcast.TorrentBroadcast.readBroadcastBlock(TorrentBroadcast.scala:206)
	at org.apache.spark.broadcast.TorrentBroadcast._value$lzycompute(TorrentBroadcast.scala:66)
	at org.apache.spark.broadcast.TorrentBroadcast._value(TorrentBroadcast.scala:66)
	at org.apache.spark.broadcast.TorrentBroadcast.getValue(TorrentBroadcast.scala:96)
	at org.apache.spark.broadcast.Broadcast.value(Broadcast.scala:70)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:86)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:53)
	at org.apache.spark.scheduler.Task.run(Task.scala:108)
	at org

In [ ]:
features.rdd.take(10)

res11: Array[org.bdgenomics.formats.avro.Feature] = Array({"featureId": "Gene.10639::c10000_g1_i1::g.10639", "name": "ORF%20type%3A5prime_partial%20len%3A112%20%28-%29%2Cscore%3D25.49%2CUniRef90_UPI000440677B%7C100.0%7C2.8e-56%2CUniRef90_UPI0009517C74%7C99.1%7C6.2e-56%2CUniRef90_L5JU14%7C99.1%7C8.1e-56%2CUniRef90_UPI0007689164%7C99.1%7C8.1e-56%2CUniRef90_UPI00042CD8A7%7C98.2%7C2.3e-55%2CUniRef90_K9KD70%7C97.3%7C3.1e-55%2CUniRef90_F7DFP8%7C97.3%7C3.1e-55%2CUniRef90_UPI0003290704%7C96.4%7C3.1e-55%2CUniRef90_UPI00033436D2%7C96.4%7C4.0e-55%2CUniRef90_UPI00057BC7E4%7C97.3%7C5.2e-55%2CUniRef90_UPI0007047ADE%7C97.3%7C5.2e-55%2CUniRef90_UPI0009453B6C%7C96.4%7C5.2e-55%2CUniRef90_E2QZ15%7C96.4%7C8.9e-55%2CUniRef90_E1BDD1%7C97.3%7C1.5e-54%2CUniRef90_G3MXA6%7C97.3%7C1.5e-54%2CUniRef90_A0A287BJY9%7C...

entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
features.rdd.map(_.getFeatureType).distinct().collect().toList

res16: List[String] = List(mRNA, CDS, three_prime_UTR, exon, gene, five_prime_UTR)


entries total 
 
 
 
 
 
 
 
 
 entries total